In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("http://minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")
df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


In [4]:
# 後の分析用にchannelをonehotに変換
df = pd.concat([df, pd.get_dummies(df["channel"], drop_first=True)], axis=1)
df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,Phone,Web
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0,1,0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0,1
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0,0,1
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,0,1
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0,0,1


In [5]:
# バイアスデータの作成

# 男性向けメールが配信されたユーザにのみ限定
male_df = df.query(" segment!= 'Womens E-Mail' ").copy()
male_df["treatment"] = (male_df["segment"]=="Mens E-Mail").astype(int)

# treatment==0の行は購入意欲のあるユーザを抽選対象にする
male_df["obs_rate_c"] = 1
male_df["obs_rate_c"] = male_df["obs_rate_c"].mask(((male_df["recency"]<6) | (male_df["history"]>300) | (male_df["channel"]=="Multichannel")), 0.5)
# treatment==1の行は購入意欲のないユーザを抽選対象にする
male_df["obs_rate_t"] = 0.5
male_df["obs_rate_t"] = male_df["obs_rate_t"].mask(((male_df["recency"]<6) | (male_df["history"]>300) | (male_df["channel"]=="Multichannel")), 1)

np.random.seed(2)
n = len(male_df)
male_df["random_number"] = np.random.rand(n)
biased_data = male_df.query(" (treatment==0 & random_number<obs_rate_c) | (treatment==1 & random_number<obs_rate_t)")

In [6]:
biased_data.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,Phone,Web,treatment,obs_rate_c,obs_rate_t,random_number
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0,1,0,0.5,1.0,0.435995
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,0,1,1,0.5,1.0,0.025926
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1,0,1,0.5,1.0,0.549662
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,0,1,1,0.5,1.0,0.435322
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,No E-Mail,0,0,0.0,0,0,0,0.5,1.0,0.420368


In [7]:
biased_data.describe()

,recency,history,mens,womens,newbie,visit,conversion,spend,Phone,Web,treatment,obs_rate_c,obs_rate_t,random_number
count,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000,31896.000000
mean,5.769156,241.797023,0.552922,0.548439,0.501379,0.148483,0.009500,1.089529,0.434882,0.444319,0.540695,0.694742,0.805258,0.416344
std,3.502657,257.656348,0.497199,0.497656,0.500006,0.355583,0.097003,15.544929,0.495749,0.496898,0.498349,0.243821,0.243821,0.277075
min,1.000000,29.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000043
25%,2.000000,64.417500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.186871
50%,6.000000,156.655000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,1.000000,0.373449
75%,9.000000,324.832500,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.625260
max,12.000000,3345.930000,1.000000,1.000000,1.000000,1.000000,1.000000,499.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.999943


In [8]:
y = biased_data[["spend"]]
x = biased_data[["history", "treatment"]]
x = sm.add_constant(x) # 切片あり

model = sm.OLS(y, x).fit()

In [9]:
print(f"coef：\n{model.params}")
print(f"P>|t|：\n{model.pvalues}")
model.summary()

coef：
const        0.326570
history      0.001279
treatment    0.839209
dtype: float64
P>|t|：
const        0.025131
history      0.000176
treatment    0.000002
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     21.26
Date:                Sat, 06 Aug 2022   Prob (F-statistic):           5.95e-10
Time:                        10:24:03   Log-Likelihood:            -1.3275e+05
No. Observations:               31896   AIC:                         2.655e+05
Df Residuals:                   31893   BIC:                         2.655e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3266      0.146      2.239      0.025       0.041       0.612
history        0.0013      0.000      3.751      0.000       0.001       0.002
treatment      0.8392      0.176      4.761      0.000       0.494       1.185
==============================================================================
Omnibus:                    70369.754   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        338405011.546
Skew:                          20.589   Prob(JB):                         0.00
Kurtosis:                     505.927   Cond. No.                         830.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### RCTデータに対して spend ~ treatmentして真の効果の値を推定する

In [10]:
y = male_df[["spend"]]
x = male_df[["treatment"]]
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()

# 真の効果量
print(f"coef：\n{model.params}")
print(f"P>|t|：\n{model.pvalues}")
model.summary()

coef：
const        0.652789
treatment    0.769827
dtype: float64
P>|t|：
const        2.093808e-10
treatment    1.163201e-07
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     28.09
Date:                Sat, 06 Aug 2022   Prob (F-statistic):           1.16e-07
Time:                        10:24:20   Log-Likelihood:            -1.7584e+05
No. Observations:               42613   AIC:                         3.517e+05
Df Residuals:                   42611   BIC:                         3.517e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6528      0.103      6.356      0.000       0.451       0.854
treatment      0.7698      0.145      5.300      0.000       0.485       1.055
==============================================================================
Omnibus:                    94877.860   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        502270597.344
Skew:                          21.023   Prob(JB):                         0.00
Kurtosis:                     533.203   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
y = biased_data[["spend"]]
x = biased_data[["treatment"]]
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()

print(f"coef：\n{model.params}")
print(f"P>|t|：\n{model.pvalues}")
model.summary()

coef：
const        0.586214
treatment    0.930867
dtype: float64
P>|t|：
const        4.980444e-06
treatment    9.786007e-08
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     28.43
Date:                Sat, 06 Aug 2022   Prob (F-statistic):           9.79e-08
Time:                        10:24:32   Log-Likelihood:            -1.3276e+05
No. Observations:               31896   AIC:                         2.655e+05
Df Residuals:                   31894   BIC:                         2.655e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5862      0.128      4.566      0.000       0.335       0.838
treatment      0.9309      0.175      5.332      0.000       0.589       1.273
==============================================================================
Omnibus:                    70370.699   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        338250637.711
Skew:                          20.590   Prob(JB):                         0.00
Kurtosis:                     505.812   Cond. No.                         2.72
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### セレクションバイアスを全て共変量に加えることができれば推定量は真の値に近づくはず

In [12]:
y = biased_data[["spend"]]
x = biased_data[["treatment", "recency", "history", "Phone", "Web"]]
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()

print(f"coef：\n{model.params}")
print(f"P>|t|：\n{model.pvalues}")
model.summary()

coef：
const        0.522710
treatment    0.784799
recency     -0.039567
history      0.001209
Phone       -0.132759
Web          0.306342
dtype: float64
P>|t|：
const        0.171817
treatment    0.000013
recency      0.130670
history      0.001462
Phone        0.665424
Web          0.317678
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     10.10
Date:                Sat, 06 Aug 2022   Prob (F-statistic):           1.10e-09
Time:                        10:25:28   Log-Likelihood:            -1.3275e+05
No. Observations:               31896   AIC:                         2.655e+05
Df Residuals:                   31890   BIC:                         2.656e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5227      0.383      1.366      0.172      -0.227       1.273
treatment      0.7848      0.180      4.353      0.000       0.431       1.138
recency       -0.0396      0.026     -1.512      0.131      -0.091       0.012
history        0.0012      0.000      3.182      0.001       0.000       0.002
Phone         -0.1328      0.307     -0.432      0.665      -0.734       0.469
Web            0.3063      0.307      0.999      0.318      -0.295       0.907
==============================================================================
Omnibus:                    70359.934   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        338144672.630
Skew:                          20.582   Prob(JB):                         0.00
Kurtosis:                     505.733   Cond. No.                     2.17e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### OVB
modelA：spend = α0 + α1*treatment + α2*recency + α3*channel  
modelB：spend = β0 + β1*treatment + β2*recency + β3*channel + β4*history  
modelC：history = γ0 + γ1*treatment + γ2*recency + γ3channel  
α1 = β1 + γ1*β4

In [13]:
yA = biased_data[["spend"]]
xA = biased_data[["treatment", "recency", "Phone", "Web"]]
xA = sm.add_constant(xA)
modelA = sm.OLS(yA, xA).fit()

yB = biased_data[["spend"]]
xB = biased_data[["treatment", "recency", "history", "Phone", "Web"]]
xB = sm.add_constant(xB)
modelB = sm.OLS(yB, xB).fit()

yC = biased_data[["history"]]
xC = biased_data[["treatment", "recency", "Phone", "Web"]]
xC = sm.add_constant(xC)
modelC = sm.OLS(yC, xC).fit()

In [21]:
print(f"α1：{modelA.params['treatment']}")
print(f"β1：{modelB.params['treatment']}")
print(f"γ1：{modelC.params['treatment']}")
print(f"β4：{modelB.params['history']}")

α1：0.8207339612315595
β1：0.7847988940903124
γ1：29.719203791976252
β4：0.0012091530914751436


In [23]:
ovb = modelC.params['treatment'] * modelB.params['history']
coef_gap = modelA.params['treatment'] - modelB.params['treatment']
print(f"OVB：{ovb}")
print(f"coef_gap：{coef_gap}")

OVB：0.035935067141247896
coef_gap：0.03593506714124706
